In [12]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO
import matplotlib.pyplot as plt

In [19]:
select = 'valid'
GT_JSON = f'../../../Other/Dongjin/1010_split/result/{select}_3_5.json'
PRED_CSV = f'./result/deta-swin-large_3_img_size_720_{select}.csv'

LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal",
              "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]




In [20]:
# load ground truth
with open(GT_JSON, 'r') as outfile:
    test_anno = (json.load(outfile))

# load prediction
pred_df = pd.read_csv(PRED_CSV)

pred = []
file_names = pred_df['image_id'].values.tolist()
bboxes = pred_df['PredictionString'].values.tolist()

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).strip().split(' '))

    if len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    elif isinstance(bbox, float):
        print(f'{file_name} empty box')
        continue
    else:
        raise Exception('error', 'invalid box type')
    
    for box in boxes:
        pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])

958it [00:00, 5201.01it/s]


In [21]:
gt = []
coco = COCO(GT_JSON)

for image_id in coco.getImgIds():
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
    file_name = image_info['file_name']

    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                float(annotation['bbox'][0]),
                float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                float(annotation['bbox'][1]),
                float(annotation['bbox'][1]) + float(annotation['bbox'][3])])

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [22]:
mean_ap, average_precisions = mean_average_precision_for_boxes(gt, pred, iou_threshold=0.5)
print(mean_ap)

Number of files in annotations: 958
Number of files in predictions: 958
Unique classes: 10
Detections length: 958
Annotations length: 958
0                              | 0.423034 |     804
1                              | 0.618705 |    1214
2                              | 0.671341 |     180
3                              | 0.648315 |     190
4                              | 0.786109 |     229
5                              | 0.583906 |     595
6                              | 0.560847 |     328
7                              | 0.765935 |     941
8                              | 0.870503 |      51
9                              | 0.744020 |      55
mAP: 0.667272
0.6672715213606737


In [26]:
def box_iou_calc(boxes1, boxes2):
    """ 
    <https://github.com/pytorch/vision/blob/master/torchvision/ops/boxes.py>
    Return intersection-over-union (Jaccard index) of boxes.
    Both sets of boxes are expected to be in (x1, y1, x2, y2) format.
    """

    def box_area(box):
        return (box[2] - box[0]) * (box[3] - box[1])
    
    area1 = box_area(boxes1.T)
    area2 = box_area(boxes2.T)

    lt = np.maximum(boxes1[:, None, :2], boxes2[:, :2]) # left top?
    rb = np.minimum(boxes1[:, None, 2:], boxes2[:, 2:]) # right bottom?

    inter = np.prod(np.clip(rb - lt, a_min=0, a_max=None), 2)
    return inter / (area1[:, None] + area2 - inter) # iou = inter / (area1 + area2 - inter)